In [34]:
# Cell 1: Enable interactive matplotlib backend
%matplotlib widget

RuntimeError: 'widget is not a recognised GUI loop or backend name

In [ ]:
# Cell 2: Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, FloatSlider

In [ ]:
# Cell 3: Demo A – two scalar‐field cosines (E & B) with sliders
def run_wave(E0=1.0, B0=0.5, phase_offset=np.pi/2):
    x = np.linspace(0, 10, 500)
    k = 2 * np.pi / 10
    omega = 2 * np.pi / 5

    fig, ax = plt.subplots()
    lineE, = ax.plot(x, E0 * np.cos(k*x),   label="E‑field")
    lineB, = ax.plot(x, B0 * np.cos(k*x + phase_offset),
                     linestyle="--", label="B‑field")
    ax.set_ylim(-1.5, 1.5)
    ax.set_xlabel("x")
    ax.set_ylabel("Amplitude")
    ax.legend()

    def update(frame):
        t = frame / 20.0
        lineE.set_ydata(E0 * np.cos(k*x - omega*t))
        lineB.set_ydata(B0 * np.cos(k*x - omega*t + phase_offset))
        return lineE, lineB

    ani = FuncAnimation(fig, update, frames=200, interval=50)
    return ani

In [ ]:
# Cell 4: Sliders for Demo A
interact(
    run_wave,
    E0=FloatSlider(min=0.0, max=2.0, step=0.1, value=1.0, description="E0"),
    B0=FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, description="B0"),
    phase_offset=FloatSlider(min=0, max=2*np.pi, step=0.1,
                             value=np.pi/2, description="Phase")
)

In [ ]:
# Cell 5: Demo B – 1D FDTD Maxwell Solver setup
Nx = 300
dx = 1.0
c  = 1.0
dt = dx / (2 * c)

Ez = np.zeros(Nx)
Hy = np.zeros(Nx)

source_position = Nx // 3

In [ ]:
# Cell 6: Demo B – initialize plot
fig_fdtd, ax_fdtd = plt.subplots()
lineE_fdtd, = ax_fdtd.plot(Ez, label="Ez")
lineH_fdtd, = ax_fdtd.plot(Hy, label="Hy", linestyle="--")
ax_fdtd.set_ylim(-1.2, 1.2)
ax_fdtd.set_xlim(0, Nx)
ax_fdtd.set_xlabel("Grid index")
ax_fdtd.set_ylabel("Field")
ax_fdtd.legend()

In [ ]:
# Cell 7: Demo B – time‐stepping + animation
def update_fdtd(n, Ez, Hy, source_pos, dt, dx):
    for i in range(Nx - 1):
        Hy[i] += (dt/dx)*(Ez[i+1] - Ez[i])
    for i in range(1, Nx):
        Ez[i] += (dt/dx)*(Hy[i] - Hy[i-1])
    Ez[source_pos] += np.sin(2 * np.pi * 0.02 * n)
    lineE_fdtd.set_ydata(Ez)
    lineH_fdtd.set_ydata(Hy)
    return lineE_fdtd, lineH_fdtd

ani_fdtd = FuncAnimation(
    fig_fdtd,
    update_fdtd,
    fargs=(Ez, Hy, source_position, dt, dx),
    frames=500,
    interval=20
)
ani_fdtd

In [ ]:
# Cell 8: Demo C – scalar wave through an E/B‐field “barrier”
def run_barrier_wave(E_amp=1.0, B_amp=0.5, center=5.0, width=1.0):
    L = 10
    Np = 200
    x = np.linspace(0, L, Np)
    dx = x[1] - x[0]

    E_profile = E_amp * np.exp(-((x - center)**2)/(2*width**2))
    B_profile = B_amp * np.exp(-((x - center)**2)/(2*width**2))
    v = 1.0 / (1.0 + E_profile + B_profile)

    dt_local = 0.5 * dx / np.max(v)
    phi = np.exp(-((x - 2.0)**2)/(2*0.2**2))
    phi_old = phi.copy()

    frames = []
    for n in range(300):
        phi_new = np.zeros_like(phi)
        cfl2 = (dt_local/dx)**2
        for i in range(1, Np-1):
            phi_new[i] = (
                2*phi[i] - phi_old[i]
                + cfl2 * v[i]**2 * (phi[i+1] - 2*phi[i] + phi[i-1])
            )
        phi_new[0] = phi_new[-1] = 0
        phi_old, phi = phi, phi_new
        if n % 2 == 0:
            frames.append(phi.copy())

    fig, ax = plt.subplots()
    line, = ax.plot(x, frames[0])
    ax.set_ylim(-1.0, 1.0)
    ax.set_xlabel("x")
    ax.set_ylabel("φ(x,t)")
    ax.set_title("Scalar wave through E/B‐field barrier")

    def animate(i):
        line.set_ydata(frames[i])
        return (line,)

    ani = FuncAnimation(fig, animate, frames=len(frames), interval=50)
    return ani

In [ ]:
# Cell 9: Sliders for Demo C
interact(
    run_barrier_wave,
    E_amp=FloatSlider(min=0.0, max=5.0, step=0.1, value=1.0, description="E_amp"),
    B_amp=FloatSlider(min=0.0, max=5.0, step=0.1, value=0.5, description="B_amp"),
    center=FloatSlider(min=0.0, max=10.0, step=0.1, value=5.0, description="Center"),
    width=FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description="Width")
)